In [17]:
import modal
from egfr_binder_rd2.utils import hash_seq
from egfr_binder_rd2 import EGFR, EGFS
import pandas as pd
from pathlib import Path
from egfr_binder_rd2 import MODAL_VOLUME_PATH, OUTPUT_DIRS

get_msa = modal.Function.lookup("simplefold", 'get_msa_for_binder')
# a3m_from_template = modal.Function.lookup("simplefold", 'a3m_from_template')
fold_binder = modal.Function.lookup("simplefold", 'fold_binder')
update_metrics = modal.Function.lookup("simplefold", 'update_metrics_for_all_folded')
esm2_pll = modal.Function.lookup("esm2-inference", 'process_sequences')
update_pll_metrics = modal.Function.lookup("esm2-inference", 'update_pll_metrics')

In [2]:
binder_seqs = [
    'WVQLQESGGGLVQPGGSLRLSCAASGRTFSSYAMGWFRQAPGKQREFVAAIRWSGGYTYYTDSVKGRFTISRDNAKTTVYLQMNSLKPEDTAVYYCAATYLSSDYSRYALPQRPLDYDYWGQGTQVTVSSLE:LEEKKVCQGTSNKLTQLGTFEDHFLSLQRMFNNCEVVLGNLEITYVQRNYDLSFLKTIQEVAGYVLIALNTVERIPLENLQIIRGNMYYENSYALAVLSNYDANKTGLKELPMRNLQEILHGAVRFSNNPALCNVESIQWRDIVSSDFLSNMSMDFQNHLGSCQKCDPSCPNGSCWGAGEENCQKLTKIICAQQCSGRCRGKSPSDCCHNQCAAGCTGPRESDCLVCRKFRDEATCKDTCPPLMLYNPTTYQMDVNPEGKYSFGATCVKKCPRNYVVTDHGSCVRACGADSYEMEEDGVRKCKKCEGPCRKVCNGIGIGEFKDSLSINATNIKHFKNCTSISGDLHILPVAFRGDSFTHTPPLDPQELDILKTVKEITGFLLIQAWPENRTDLHAFENLEIIRGRTKQHGQFSLAVVSLNITSLGLRSLKEISDGDVIISGNKNLCYANTINWKKLFGTSGQKTKIISNRGENSCKATGQVCHALCSPEGCWGPEPRDCVSCRNVSRGRECVDKCKLLEGEPREFVENSECIQCHPECLPQAMNITCTGRGPDNCIQCAHYIDGPHCVKTCPAGVMGENNTLVWKYADAGHVCHLCHPNCTYGCTGPGLRGCPTNGHHHHHH',
    'QVQLQESGGGLVQPGGSLRLSCAASGRTFSSHAMGWFRQAPGKQREFVAAIRWSGGYTYYTDSVKGRFTISRDNAKTTVYLQMNSLKPEDTAVYYCAATYLSSDYSRYALPQRPLDYDYWGQGTQVTVSSLE:LEEKKVCQGTSNKLTQLGTFEDHFLSLQRMFNNCEVVLGNLEITYVQRNYDLSFLKTIQEVAGYVLIALNTVERIPLENLQIIRGNMYYENSYALAVLSNYDANKTGLKELPMRNLQEILHGAVRFSNNPALCNVESIQWRDIVSSDFLSNMSMDFQNHLGSCQKCDPSCPNGSCWGAGEENCQKLTKIICAQQCSGRCRGKSPSDCCHNQCAAGCTGPRESDCLVCRKFRDEATCKDTCPPLMLYNPTTYQMDVNPEGKYSFGATCVKKCPRNYVVTDHGSCVRACGADSYEMEEDGVRKCKKCEGPCRKVCNGIGIGEFKDSLSINATNIKHFKNCTSISGDLHILPVAFRGDSFTHTPPLDPQELDILKTVKEITGFLLIQAWPENRTDLHAFENLEIIRGRTKQHGQFSLAVVSLNITSLGLRSLKEISDGDVIISGNKNLCYANTINWKKLFGTSGQKTKIISNRGENSCKATGQVCHALCSPEGCWGPEPRDCVSCRNVSRGRECVDKCKLLEGEPREFVENSECIQCHPECLPQAMNITCTGRGPDNCIQCAHYIDGPHCVKTCPAGVMGENNTLVWKYADAGHVCHLCHPNCTYGCTGPGLRGCPTNGHHHHHH',
    'SVDEECPASYEGFCQNDGTCLYLEKLDRYACRCREGYIGERCEFRDLDYWLEQ',
    'DSECPLSHDGYCLHDGVCMYIEALDKYACNCVVGYIGERCQYRDLKWWENLEERLKEHRAKRLALLGPGPPGVVEKEKYKVSITEKVNPGGPATMPMTLTDSNGNKTTLTITVTPEGLEAIRKRRAGEKVKYTMTSTDTGDKFVLVDLDGYCLHDGVCMYIEALDKYACNCVVGYIGERCQYRDLK',
    'DSECPLSHDGYCLHDGVCMYIEALDKYACNCVVGYIGERCQYRDLKWWENLEERLKEHRAKRLALL',
    'NSDSECPLSHDGYCLHDGVCMYIEALDKYACNCVVGYIGERCQYRDLKWWELR',
    'AERMRRRFEHIVEIHEEWAKEVLENLKKQGSKEEDLKFMEEYLEQDVEELRKRAEEMVEEYEKSS',
]


In [9]:
parent_binder_seq = 'PSFSACPSNYDGYCMNGGVCHYFESLTSITCQCIIGYIGDRCQTFDLRYTELRR'
binder_seqs = ['PSFSACPSNYDGYCMNGGVCHYFESLTSITCQCIIGYIGDRCQTFDDRYTELRR']

result = fold_binder.remote(binder_seqs=binder_seqs, parent_binder_seqs=[parent_binder_seq])

In [ ]:

# binder_seqs = ['AERMRRRFEHIVEIHEEWAKEVLENLKKQGSKEEDLKFMEEYLEQDVEELRKRAEEMVEEYEKSS']

# result = get_msa.remote(binder_seqs)


In [20]:
result = esm2_pll.remote()

In [3]:
result = esm2_pll.remote(binder_seqs)

In [10]:
parent_binder_seq = 'SKEEEYYEEHQKLAKPVEELWEKLDELEKTGKLTGEHRPLVTEFRRLWSDAMVLIAMYMWYLEEVDKNPSEENRKKAQEYLEKVEEKKKEMEELLKKL'
parent_binder_seq = 'PSFSACPSNYDGYCMNGGVCHYFESLTSITCQCIIGYIGDRCQTFDLRYTELR'

In [15]:
parent_binder_seq = 'PSFSACPSNYDGVCCNGGVCHLAESLTSYTCQCILGYSGHRVQTFDLRYTELRRR'

In [16]:
result = get_msa.remote([parent_binder_seq])

In [18]:
df = update_pll_metrics.remote()
fdf = update_metrics.remote()
fdf = fdf.merge(df, left_on='binder_sequence', right_on='sequence', how='left')
fdf['pae_interaction_rank'] = 1 - fdf['pae_interaction'].rank(pct=True)
fdf['i_ptm_rank'] = fdf['i_ptm'].rank(pct=True)
fdf['sequence_log_pll_rank'] = fdf['sequence_log_pll'].rank(pct=True)
fdf['fitness'] = (fdf['pae_interaction_rank'] + fdf['i_ptm_rank'] + fdf['sequence_log_pll_rank']) / 3
fdf = fdf.sort_values('fitness', ascending=False).reset_index(drop=True)

In [19]:
pd.set_option('display.max_columns', None)
fdf

,seq_hash,binder_sequence,binder_length,target_sequence,target_length,model_number,binder_plddt,binder_pae,pae_interaction,ptm,i_ptm,binder_charged_fraction,binder_hydrophobic_fraction,parent_hash,parent_sequence,mutations,sequence_hash,sequence,sequence_length,normalized_log_pll,sequence_log_pll,mean_token_probability,min_token_probability,pae_interaction_rank,i_ptm_rank,sequence_log_pll_rank,fitness
0,cea805,PSFSACPSNYDGVCCNGGVCHLAESLTSYTCQCALGYSGPRCQTLD...,55,LEEKKVCQGTSNKLTQLGTFEDHFLSLQRMFNNCEVVLGNLEITYV...,622,1,85.980545,5.391134,8.984942,0.86,0.91,0.181818,0.236364,f2b3c7,PSFSACPSNYDGVCCNGGVCHLAESLTSYTCQCILGYSGHRVQTFD...,"I34A,H40P,V42C,F45L,Y49V",bdr_a69753,PSFSACPSNYDGVCCNGGVCHLAESLTSYTCQCALGYSGPRCQTLD...,57.0,-3.092197,-176.255249,0.785187,0.013342,0.958855,0.919499,0.805882,0.894746
1,5b353a,NSYPGCPSSYDGYCLNGGVCMHIESLDSYTCNCVIGYSGDRCQTRD...,53,LEEKKVCQGTSNKLTQLGTFEDHFLSLQRMFNNCEVVLGNLEITYV...,622,1,85.922264,5.186330,8.895448,0.86,0.92,0.207547,0.301887,NaN,NaN,NaN,bdr_a07fd0,NSYPGCPSSYDGYCLNGGVCMHIESLDSYTCNCVIGYSGDRCQTRD...,55.0,-4.331452,-238.229858,0.816994,0.190863,0.971377,0.961538,0.711765,0.881560
2,f35f98,ISFSACPSNYDGVCCNGGVCHLAESLTSYTCICRLGYSGHRCQTVD...,55,LEEKKVCQGTSNKLTQLGTFEDHFLSLQRMFNNCEVVLGNLEITYV...,622,1,86.228364,5.029312,8.837427,0.86,0.91,0.218182,0.272727,f2b3c7,PSFSACPSNYDGVCCNGGVCHLAESLTSYTCQCILGYSGHRVQTFD...,"P1I,Q32I,I34R,V42C,F45V",bdr_4144ee,ISFSACPSNYDGVCCNGGVCHLAESLTSYTCICRLGYSGHRCQTVD...,57.0,-3.796905,-216.423599,0.772350,0.020983,0.973166,0.919499,0.749020,0.880562
3,911e2a,PSFSACPSLYDGVCCNGGVCHLAESLTSYTCQCHPGYSGHRCQTVD...,55,LEEKKVCQGTSNKLTQLGTFEDHFLSLQRMFNNCEVVLGNLEITYV...,622,1,87.206909,4.827035,8.651548,0.86,0.92,0.218182,0.236364,f2b3c7,PSFSACPSNYDGVCCNGGVCHLAESLTSYTCQCILGYSGHRVQTFD...,"N9L,I34H,L35P,V42C,F45V",bdr_45ee65,PSFSACPSLYDGVCCNGGVCHLAESLTSYTCQCHPGYSGHRCQTVD...,57.0,-4.399897,-250.794128,0.767408,0.029103,0.983900,0.961538,0.682353,0.875930
4,744634,PSFSACPSNYDGVCSNGGVCHLAESLTSYTCQCQLGYSGHRVQTFD...,55,LEEKKVCQGTSNKLTQLGTFEDHFLSLQRMFNNCEVVLGNLEITYV...,622,1,87.092364,4.846179,8.762531,0.86,0.92,0.200000,0.254545,f2b3c7,PSFSACPSNYDGVCCNGGVCHLAESLTSYTCQCILGYSGHRVQTFD...,"C15S,I34Q",bdr_b0fd0b,PSFSACPSNYDGVCSNGGVCHLAESLTSYTCQCQLGYSGHRVQTFD...,57.0,-4.556254,-259.706482,0.753190,0.006044,0.976744,0.961538,0.670588,0.869624
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
554,b05bc1,PMKSACPSAYDGYCRNGGVCHYAESLTSYTCDCIIGYIGDRCQTFD...,54,LEEKKVCQGTSNKLTQLGTFEDHFLSLQRMFNNCEVVLGNLEITYV...,622,1,81.112037,5.900257,10.164033,0.83,0.87,0.259259,0.240741,8d755b,PSFSACPSNYDGYCRNGGVCHYAESLTSYTCDCIIGYIGDRCQTFD...,"S2M,F3K,N9A",NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.865832,0.705725,NaN,NaN
555,b86027,PSFSACPCNYDGVCQNGICIHLAESLTSYTCQCPLGYSGPRCQTFD...,55,LEEKKVCQGTSNKLTQLGTFEDHFLSLQRMFNNCEVVLGNLEITYV...,622,1,75.426182,7.743253,11.229122,0.84,0.87,0.163636,0.272727,595d0b,PSFSACPSNYDGVCQNGGVCHLAESLTSYTCQCPLGYSGPRCQTFD...,"S8C,G18I,V19C,C20I,R55Y",NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.856887,0.705725,NaN,NaN
556,5ba9bc,NCYPGCPSSYDGYCLNGGVCMHIESLDSYRCNCVPGYSGDRCQTRD...,53,LEEKKVCQGTSNKLTQLGTFEDHFLSLQRMFNNCEVVLGNLEITYV...,622,1,84.364151,5.329747,9.522547,0.84,0.89,0.226415,0.264151,7b1d95,NCYPGCPSSYDGYCLNGGVCMHIESLDSYTCNCVIGYSGDRCQTRD...,"T30R,I35P",NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.898032,0.778175,NaN,NaN
557,7aefd2,NMYPGCPSRYDGYCLNGGVCMHIQSLGSYTCDCVRGYSGDRCQTRD...,53,LEEKKVCQGTSNKLTQLGTFEDHFLSLQRMFNNCEVVLGNLEITYV...,622,1,82.633962,5.645732,9.956847,0.83,0.89,0.226415,0.283019,6b975d,NMYPGCPSSYDGYCLNGGVCMHIESLDSYTCNCVRGYSGDRCQTRD...,"S9R,E24Q,D27G,N32D,A49C",NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.871199,0.778175,NaN,NaN


In [7]:
fdf.query('seq_hash == "830e19"')

,seq_hash,binder_sequence,binder_length,target_sequence,target_length,model_number,binder_plddt,binder_pae,pae_interaction,ptm,i_ptm,binder_charged_fraction,binder_hydrophobic_fraction,parent_hash,parent_sequence,mutations,sequence_hash,sequence,sequence_length,normalized_log_pll,sequence_log_pll,mean_token_probability,min_token_probability,pae_interaction_rank,i_ptm_rank,sequence_log_pll_rank,fitness
23,830e19,SEEEKKKRIEELKKEYKELIDIQEKRIKEAQEEISKLDLSNEKDKQ...,80,LEEKKVCQGTSNKLTQLGTFEDHFLSLQRMFNNCEVVLGNLEITYV...,622,1,82.80075,3.589772,13.443408,0.67,0.91,0.525,0.2875,NaN,NaN,NaN,bdr_8b0068,SEEEKKKRIEELKKEYKELIDIQEKRIKEAQEEISKLDLSNEKDKQ...,82,-4.624028,-379.170288,0.769584,0.042618,0.959811,0.953901,0.404255,0.772656


In [14]:
fdf.sort_values('fitness', ascending=False).reset_index(drop=True).head(40).loc[10]

seq_hash                                                                  46ddaa
binder_sequence                PSFSACPSNYDGYCMNGGVCHYFESLTSITCQCIIGYIGDRCQTDD...
binder_length                                                                 54
target_sequence                LEEKKVCQGTSNKLTQLGTFEDHFLSLQRMFNNCEVVLGNLEITYV...
target_length                                                                622
model_number                                                                   1
binder_plddt                                                            84.52463
binder_pae                                                              5.046159
pae_interaction                                                         9.499823
ptm                                                                         0.83
i_ptm                                                                        0.9
binder_charged_fraction                                                 0.203704
binder_hydrophobic_fraction 

In [27]:
old_df = pd.read_csv('/home/naka/code/egfr_binder_rd2/data/fold_df.csv', index_col=0)

In [31]:
df.merge(old_df, how='outer')

,seq_hash,binder_sequence,binder_length,target_sequence,target_length,model_number,binder_plddt,binder_pae,pae_interaction,ptm,...,parent_hash,parent_sequence,mutations,seq_name,seq_id,pdb_content,generation,filename,mut_str,n_mutations
0,1d58b4,AERMRRRCEHIVEILEEWAKETLEELKKQGVKEEDLKFMEEYLEQD...,65,LEEKKVCQGTSNKLTQLGTFEDHFLSLQRMFNNCEVVLGNLEITYV...,622,1,71.498000,4.290866,17.108837,0.58,...,f9aee7,AERMRRRFEHIVEIHEEWAKEVLENLKKQGSKEEDLKFMEEYLEQD...,"F8C,H15L,V22T,N25E,S31V",NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,44b41e,AERMRRRFEGIVELHEEWAKETLENLKKQGSKEEDLKFMEEYLEQD...,65,LEEKKVCQGTSNKLTQLGTFEDHFLSLQRMFNNCEVVLGNLEITYV...,622,1,47.834769,5.407316,22.381617,0.62,...,f9aee7,AERMRRRFEHIVEIHEEWAKEVLENLKKQGSKEEDLKFMEEYLEQD...,"H10G,I14L,V22T",NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,dc33cd,AERMRRRFEHIQEIHEEWAKEFLENLKKQGSKEEDLKFMEEYLEQD...,65,LEEKKVCQGTSNKLTQLGTFEDHFLSLQRMFNNCEVVLGNLEITYV...,622,1,73.961385,4.464473,16.605817,0.58,...,f9aee7,AERMRRRFEHIVEIHEEWAKEVLENLKKQGSKEEDLKFMEEYLEQD...,"V12Q,V22F,K63F",NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,f9aee7,AERMRRRFEHIVEIHEEWAKEVLENLKKQGSKEEDLKFMEEYLEQD...,65,LEEKKVCQGTSNKLTQLGTFEDHFLSLQRMFNNCEVVLGNLEITYV...,622,1,79.039538,3.777522,16.788581,0.58,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,6f4ce9,AERMRRRVERIVEIHEEEAKEVLENLKKQGMKEEDLKCMEEYLEQD...,65,LEEKKVCQGTSNKLTQLGTFEDHFLSLQRMFNNCEVVLGNLEITYV...,622,1,42.019231,7.973046,25.950806,0.60,...,f9aee7,AERMRRRFEHIVEIHEEWAKEVLENLKKQGSKEEDLKFMEEYLEQD...,"F8V,H10R,W18E,S31M,F38C",NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
893,NaN,TSYSGCPSSYDGSCGNGGVCEHIESLDSYTKQCVRGYSGDRVQTAD...,50,LEEKKVCQGTSNKLTQLGTFEDHFLSLQRMFNNCEVVLGNLEITYV...,622,1,79.396400,5.217940,10.732721,0.80,...,NaN,NaN,NaN,e581d7,e581d7,MODEL 1 ...,2.0,fold_results_20240818_161100.csv,"N1T,P4S,Y13S,L15G,M21E,C31K,N32Q,I35R,C42V,R45...",11.0
894,NaN,VSYFACNSSYAGYCLESGVCQHIESLDSYTCHCVIGYSGDRTQTHD...,50,LEEKKVCQGTSNKLTQLGTFEDHFLSLQRMFNNCEVVLGNLEITYV...,622,1,90.409800,3.445576,8.121858,0.85,...,NaN,NaN,NaN,f4c5a2,f4c5a2,MODEL 1 ...,2.0,fold_results_20240818_224107.csv,"N1V,P4F,G5A,P7N,D11A,N16E,G17S,M21Q,N32H,C42T,...",13.0
895,NaN,VSYSACPSSYDGYCGNGGVCKHIESLDSYTCQCVIGYSGDRVQTAD...,50,LEEKKVCQGTSNKLTQLGTFEDHFLSLQRMFNNCEVVLGNLEITYV...,622,1,88.763800,3.700192,8.788995,0.83,...,NaN,NaN,NaN,069a2f,069a2f,MODEL 1 ...,2.0,fold_results_20240818_162704.csv,"N1V,P4S,G5A,L15G,M21K,N32Q,C42V,R45A,R48P,W49Y...",11.0
896,NaN,WFYSACPSSYDGYCGNGGVCMHIQSLDSYTCRCYIGYSGPRVQTRD...,50,LEEKKVCQGTSNKLTQLGTFEDHFLSLQRMFNNCEVVLGNLEITYV...,622,1,89.079000,3.756444,8.364633,0.85,...,NaN,NaN,NaN,29d172,29d172,MODEL 1 ...,3.0,fold_results_20240818_225601.csv,"N1W,S2F,P4S,G5A,L15G,E24Q,N32R,V34Y,D40P,C42V,...",12.0


In [13]:
parent_binder_seq = 'AERMRRRFEHIVEIHEEWAKEVLENLKKQGSKEEDLKFMEEYLEQDVEELRKRAEEMVEEYEKSS'

In [7]:
result = fold_binder.remote(binder_seqs=[parent_binder_seq])

In [9]:

result = fold_binder.remote(binder_seq='SALDEWLASLDPQVGQDIRDYIEERQAE', parent_binder_seq=parent_binder_seq)

In [44]:
seqs = fdf['binder_sequence'].tolist()

In [45]:
result = esm2_pll.remote(seqs)


In [50]:
sequence = 'PSFSACPSNYDGYCMNGGVCHYFESLTSITCQCIIGYIGDRCQTFDLRYTELRR'
result = get_msa.remote([sequence])
print(f"MSA result: {result}")

MSA result: [PosixPath('/colabfold_data/msa_results/f074a3.a3m')]


In [51]:
result = fold_binder.remote(binder_seqs=[sequence])
print(f"Folding result: {result}")

Folding result: [None]


In [52]:
df = update_metrics.remote()
# Filter for just our sequence
seq_hash = hash_seq(f"{sequence}:{EGFR}")
sequence_results = df[df['seq_hash'] == seq_hash]
print(sequence_results)

   seq_hash                                    binder_sequence  binder_length  \
86   f074a3  PSFSACPSNYDGYCMNGGVCHYFESLTSITCQCIIGYIGDRCQTFD...             54   

                                      target_sequence  target_length  \
86  LEEKKVCQGTSNKLTQLGTFEDHFLSLQRMFNNCEVVLGNLEITYV...            622   

    model_number  binder_plddt  binder_pae  pae_interaction   ptm  i_ptm  \
86             1     85.282037    5.009801         9.262812  0.84    0.9   

    binder_charged_fraction  binder_hydrophobic_fraction parent_hash  \
86                 0.185185                     0.314815        None   

   parent_sequence mutations  
86            None      None  


In [2]:
parent_sequence = 'PSFSACPSNYDGYCMNGGVCHYFESLTSITCQCIIGYIGDRCQTFDLRYTELRR'
mutant_seqs = [
    # 'PSFSACPSNYDGYCMNGGVCHYFESLTSITCQCIIGYIGDRCQTFDLRYTELRK',
    # 'ASFSACPSNYDGYCMNGGVCHYFESLTSITCQCIIGYIGDRCQTFDLRYTELRR',
    'PSFSACPSNYDGYCMNGGVCHYFESLTSITCQCIIGYIGDRCQTFDLRYTELRA',
]

# Generate a3ms using the parent as template
result = a3m_from_template.remote(
    binder_seqs=mutant_seqs,
    parent_binder_seqs=[parent_sequence] * len(mutant_seqs)
)
print(f"Generated a3ms: {result}")


Generated a3ms: [PosixPath('/colabfold_data/msa_results/09f02f.a3m')]


In [3]:
# Try folding one of the mutants
result = fold_binder.remote(
    binder_seqs=[mutant_seqs[0]], 
    parent_binder_seqs=[parent_sequence]
)
print(f"Folding result: {result}")

Folding result: [None]


In [6]:
df = update_metrics.remote()

In [7]:
df

,seq_hash,binder_sequence,binder_length,target_sequence,target_length,model_number,binder_plddt,binder_pae,pae_interaction,ptm,i_ptm,binder_charged_fraction,binder_hydrophobic_fraction,parent_hash,parent_sequence,mutations
0,5b353a,NSYPGCPSSYDGYCLNGGVCMHIESLDSYTCNCVIGYSGDRCQTRD...,53,LEEKKVCQGTSNKLTQLGTFEDHFLSLQRMFNNCEVVLGNLEITYV...,622,1,85.922264,5.186330,8.895448,0.86,0.92,0.207547,0.301887,NaN,NaN,NaN
1,5b353a,NSYPGCPSSYDGYCLNGGVCMHIESLDSYTCNCVIGYSGDRCQTRD...,53,LEEKKVCQGTSNKLTQLGTFEDHFLSLQRMFNNCEVVLGNLEITYV...,622,2,84.365472,5.574172,9.162459,0.86,0.91,0.207547,0.301887,NaN,NaN,NaN
2,5b353a,NSYPGCPSSYDGYCLNGGVCMHIESLDSYTCNCVIGYSGDRCQTRD...,53,LEEKKVCQGTSNKLTQLGTFEDHFLSLQRMFNNCEVVLGNLEITYV...,622,3,85.697547,5.311627,9.028248,0.85,0.91,0.207547,0.301887,NaN,NaN,NaN
3,5b353a,NSYPGCPSSYDGYCLNGGVCMHIESLDSYTCNCVIGYSGDRCQTRD...,53,LEEKKVCQGTSNKLTQLGTFEDHFLSLQRMFNNCEVVLGNLEITYV...,622,4,84.229811,5.659277,9.556460,0.85,0.90,0.207547,0.301887,NaN,NaN,NaN
4,5b353a,NSYPGCPSSYDGYCLNGGVCMHIESLDSYTCNCVIGYSGDRCQTRD...,53,LEEKKVCQGTSNKLTQLGTFEDHFLSLQRMFNNCEVVLGNLEITYV...,622,5,82.960566,5.779459,9.915783,0.83,0.90,0.207547,0.301887,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97,7f370b,AERMRRRFEHIVEIHEEWAKEVLENLKKQGSKEEDLKFMEEYLEQD...,65,LEEKKVCQGTSNKLTQLGTFEDHFLSLQRMFNNCEVVLGNLEITYV...,622,1,78.068462,3.925882,16.826628,0.58,0.89,0.538462,0.307692,f9aee7,AERMRRRFEHIVEIHEEWAKEVLENLKKQGSKEEDLKFMEEYLEQD...,E62L
98,311b7d,AERMRRRFEHIVEIHEEWAKEVLENLKKQGSKEEDLKFMEEYLEQD...,65,LEEKKVCQGTSNKLTQLGTFEDHFLSLQRMFNNCEVVLGNLEITYV...,622,1,79.213538,3.750357,16.305776,0.59,0.90,0.538462,0.292308,f9aee7,AERMRRRFEHIVEIHEEWAKEVLENLKKQGSKEEDLKFMEEYLEQD...,K52S
99,29323a,AERMRRRFEHIVEIHEEWIKEVLENLKKQGSKEEDLKFMEEYLEQD...,65,LEEKKVCQGTSNKLTQLGTFEDHFLSLQRMFNNCEVVLGNLEITYV...,622,1,73.311385,4.341101,17.142139,0.58,0.84,0.553846,0.307692,f9aee7,AERMRRRFEHIVEIHEEWAKEVLENLKKQGSKEEDLKFMEEYLEQD...,A19I
100,6ea613,AERMRRRFEHIVENHEEWAKEVLENLKKQGSKEEDLKFMEEYLEQD...,65,LEEKKVCQGTSNKLTQLGTFEDHFLSLQRMFNNCEVVLGNLEITYV...,622,1,76.807385,3.560249,17.408015,0.57,0.87,0.553846,0.276923,f9aee7,AERMRRRFEHIVEIHEEWAKEVLENLKKQGSKEEDLKFMEEYLEQD...,I14N


In [64]:
df.query(f"binder_sequence == '{mutant_seqs[0]}'")

,seq_hash,binder_sequence,binder_length,target_sequence,target_length,model_number,binder_plddt,binder_pae,pae_interaction,ptm,i_ptm,binder_charged_fraction,binder_hydrophobic_fraction,parent_hash,parent_sequence,mutations


In [71]:
binder_seq = mutant_seqs[0]

In [72]:
template_a3m_path = '/home/naka/code/egfr_binder_rd2/src/egfr_binder_rd2/msa_results/f074a3.a3m'
output_path = 'test.a3m'
swap_binder_seq_into_a3m(binder_seq, template_a3m_path, output_path)

'test.a3m'

In [73]:
df

,seq_hash,binder_sequence,binder_length,target_sequence,target_length,model_number,binder_plddt,binder_pae,pae_interaction,ptm,i_ptm,binder_charged_fraction,binder_hydrophobic_fraction,parent_hash,parent_sequence,mutations
0,5b353a,NSYPGCPSSYDGYCLNGGVCMHIESLDSYTCNCVIGYSGDRCQTRD...,53,LEEKKVCQGTSNKLTQLGTFEDHFLSLQRMFNNCEVVLGNLEITYV...,622,1,85.922264,5.186330,8.895448,0.86,0.92,0.207547,0.301887,NaN,NaN,NaN
1,5b353a,NSYPGCPSSYDGYCLNGGVCMHIESLDSYTCNCVIGYSGDRCQTRD...,53,LEEKKVCQGTSNKLTQLGTFEDHFLSLQRMFNNCEVVLGNLEITYV...,622,2,84.365472,5.574172,9.162459,0.86,0.91,0.207547,0.301887,NaN,NaN,NaN
2,5b353a,NSYPGCPSSYDGYCLNGGVCMHIESLDSYTCNCVIGYSGDRCQTRD...,53,LEEKKVCQGTSNKLTQLGTFEDHFLSLQRMFNNCEVVLGNLEITYV...,622,3,85.697547,5.311627,9.028248,0.85,0.91,0.207547,0.301887,NaN,NaN,NaN
3,5b353a,NSYPGCPSSYDGYCLNGGVCMHIESLDSYTCNCVIGYSGDRCQTRD...,53,LEEKKVCQGTSNKLTQLGTFEDHFLSLQRMFNNCEVVLGNLEITYV...,622,4,84.229811,5.659277,9.556460,0.85,0.90,0.207547,0.301887,NaN,NaN,NaN
4,5b353a,NSYPGCPSSYDGYCLNGGVCMHIESLDSYTCNCVIGYSGDRCQTRD...,53,LEEKKVCQGTSNKLTQLGTFEDHFLSLQRMFNNCEVVLGNLEITYV...,622,5,82.960566,5.779459,9.915783,0.83,0.90,0.207547,0.301887,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
82,588e06,AERMRRRFEHIVEIYEERAKEVLENLKKQGSLEEDLKKMEEYLEQD...,65,LEEKKVCQGTSNKLTQLGTFEDHFLSLQRMFNNCEVVLGNLEITYV...,622,1,43.294769,7.995550,24.612031,0.62,0.29,0.553846,0.292308,f9aee7,AERMRRRFEHIVEIHEEWAKEVLENLKKQGSKEEDLKFMEEYLEQD...,"H15Y,W18R,K32L,F38K"
83,b7f769,AERMRRRFEWIVEIHEEWAKEVLENLKKQGIKEEDLKFMEEYLEKD...,65,LEEKKVCQGTSNKLTQLGTFEDHFLSLQRMFNNCEVVLGNLEITYV...,622,1,50.473538,5.019782,20.775382,0.63,0.46,0.553846,0.323077,f9aee7,AERMRRRFEHIVEIHEEWAKEVLENLKKQGSKEEDLKFMEEYLEQD...,"H10W,S31I,Q45K"
84,66b6f8,AERMRRRFEHIVERHEERAKEVLENLKKQGAKEEDLKFMEEYLKQD...,65,LEEKKVCQGTSNKLTQLGTFEDHFLSLQRMFNNCEVVLGNLEITYV...,622,1,47.791385,5.725243,23.921706,0.60,0.33,0.584615,0.261538,f9aee7,AERMRRRFEHIVEIHEEWAKEVLENLKKQGSKEEDLKFMEEYLEQD...,"I14R,W18R,S31A,E44K"
85,7ac155,MERMRRRFEKIVEIHEEWAKEVLEELKKQGSKEEDLKKMEEYLEQD...,65,LEEKKVCQGTSNKLTQLGTFEDHFLSLQRMFNNCEVVLGNLEITYV...,622,1,43.356154,7.629155,25.292428,0.61,0.27,0.584615,0.292308,f9aee7,AERMRRRFEHIVEIHEEWAKEVLENLKKQGSKEEDLKFMEEYLEQD...,"A1M,H10K,N25E,F38K,R53K"
